In [1]:
#Agustin Lopez
#Creating a simple Convolutional Neural Network to prepare to make the GAN 19 in the future
%matplotlib inline

Below is importing the training and testing data into my google drive

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google'

In [ ]:
%cd /content/gdrive/MyDrive/Colab Notebooks/SimpleCNNData

In [ ]:
#This downloads our test data 
repo_url = 'https://github.com/nickmccullum/cats-and-dogs'
import os
%cd /content/gdrive/MyDrive/Colab Notebooks/SimpleCNNData
repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))
!git clone {repo_url}
%cd {repo_dir_path}
!git pull

Beginning of the actual CNN

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
training_data = '/content/gdrive/MyDrive/Colab Notebooks/SimpleCNNData/cats-and-dogs/training_data'
test_data = '/content/gdrive/MyDrive/Colab Notebooks/SimpleCNNData/cats-and-dogs/test_data'

In [ ]:
#To avoid overfitting we have to do the following to our training and testing images,
#rotations 
#cropping
#Zooming
#When performed its called image augmentation

#The following method is able to apply a number of different transformations 
training_generator = ImageDataGenerator(
    rescale = 1 / 255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True)

In [ ]:
#We apply the image transformations to our training data
training_set = training_generator.flow_from_directory(
    training_data,
    target_size = (64,64),
    batch_size = 32,
    class_mode = 'binary'
    )

In [ ]:
#We create a new ImageDataGenerator because we don't want our test data to be changed
test_generator = ImageDataGenerator(rescale = 1./255)

test_set = test_generator.flow_from_directory(
    test_data,
    target_size = (64, 64),
    batch_size = 32,
    class_mode = 'binary')

In [ ]:
#Starting convolutional neural network 
#Sequential allows for a group of linear stack of layers
cnn = tf.keras.models.Sequential()

In [ ]:
#First layers - Convolutional layers
#Explanation for each parameter
#filters - the number of feature detectors we want to use in our CNN
#kernal - the size of the feature detector can be a 2x2, 3x3..., nxn
#activation - activation function
#input shape - shape of the images the last number in the array represents clolor
cnn.add(tf.keras.layers.Conv2D(
    filters=32,
    kernel_size=3,
    activation='relu',
    input_shape=[64,64,3]))

In [ ]:
#Second Layer - Max pooling layer
#Explanation for each parameter
#pool_size - the size of the matrix that views the image either 2x2, 3x3..., nxn
#strides - how far should the matrix move after one calculation

cnn.add(tf.keras.layers.MaxPool2D(
    pool_size=2,
    strides=2
))

In [ ]:
#Third Layer - Convolutional layers
cnn.add(tf.keras.layers.Conv2D(
    filters=32,
    kernel_size=3,
    activation='relu'))

In [ ]:
#Fourth Layer - Max pooling layer
cnn.add(tf.keras.layers.MaxPool2D(
    pool_size=2,
    strides=2))

In [ ]:
#Fifth Layer - Flattening layer 
cnn.add(tf.keras.layers.Flatten())

In [ ]:
#Sixth layer - Connection Layer
#we do this with another layer
cnn.add(tf.keras.layers.Dense(
    units=128, 
    activation='relu'))

In [ ]:
#seventh layer - output layer
cnn.add(tf.keras.layers.Dense(
    units=1,
    activation='sigmoid'))

Train the convolutional neural network

In [ ]:
#we need the following to compile a CNN
#optimizer
#loss function 
#some metrics
#using adams optimizer because the tutorial said so
#using stochastic for the same reason and for accuracy too
cnn.compile(optimizer = 'adam',
            loss= 'binary_crossentropy',
            metrics = ['accuracy'])


In [ ]:
#fitting the model to our training data
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Time for prediction!!!

In [ ]:
from tensorflow.keras.preprocessing import image

In [ ]:
test_image_1 = image.load_img('/content/gdrive/MyDrive/Colab Notebooks/SimpleCNNData/cats-and-dogs/predictions/cat_or_dog_1.jpg', target_size = (64, 64))

test_image_2 = image.load_img('/content/gdrive/MyDrive/Colab Notebooks/SimpleCNNData/cats-and-dogs/predictions/cat_or_dog_2.jpg', target_size= (64,64))

In [ ]:
test_image_1

In [ ]:
test_image_2

In [ ]:
#Since we grouped our training data to batches of size 32,
#we must then match this batch size to calculate our test data
test_image_1 = np.expand_dims(test_image_1, axis = 0)
test_image_2 = np.expand_dims(test_image_2, axis = 0)

In [ ]:
print("Predict the value for image 1: ", cnn.predict(test_image_1))
print("Predict the value for image 2: ", cnn.predict(test_image_2))


In [ ]:
#What did those values before mean?
training_set.class_indices

In [ ]:
#Categorical Predictions
result_1 = cnn.predict(test_image_1)

result_2 = cnn.predict(test_image_2)

if (result_1 >= 0.5):

    result_1 = 'dog'

else:

    result_1 = 'cat'

    

if (result_2 >= 0.5):

    result_2 = 'dog'

else:

    result_2 = 'cat'

print(result_1)

print(result_2)

In [ ]:
cnn.summary()